In [29]:
# !pip install yfinance
# !pip install TA-Lib
# !pip install numpy
# !pip install pandas
# !pip install vectorbt
# !pip install scipy
# %pip install scikit-optimize
# %pip install optuna


In [30]:
import yfinance as yf
import talib
import numpy as np
import pandas as pd
import vectorbt as vbt
import warnings
from scipy import stats
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", message="Degrees of freedom <= 0 for slice", category=RuntimeWarning)
warnings.filterwarnings("ignore", message="invalid value encountered in scalar divide", category=RuntimeWarning)


In [ ]:
# DOWNLOAD STOCK DATA FROM 2015 USING YFINANCE

TICKER = 'TQQQ'
START_DATE = '2018-01-01'
TRAIN_RATIO = 0.6

# Download data
stock_data = yf.download(TICKER, start=START_DATE, interval='1d')

if not stock_data.empty:
    print(f"Successfully downloaded {len(stock_data)} records for {TICKER}")
    print(f"Data range: {stock_data.index.min().date()} to {stock_data.index.max().date()}")
else:
    print(f"Failed to download {TICKER} data")

stock_data


C:\Users\harka\AppData\Local\Temp\ipykernel_13800\3709481360.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Successfully downloaded 3952 records for TQQQ
Data range: 2010-02-11 to 2025-10-27


Price,Close,High,Low,Open,Volume
Ticker,TQQQ,TQQQ,TQQQ,TQQQ,TQQQ
Date,,,,,
2010-02-11,0.413438,0.415679,0.387652,0.388896,3456000
2010-02-12,0.415131,0.418715,0.399848,0.402187,8601600
2010-02-16,0.431211,0.432207,0.418217,0.424888,9619200
2010-02-17,0.438528,0.438628,0.430414,0.436986,19180800
2010-02-18,0.446842,0.449480,0.435442,0.438080,38860800
...,...,...,...,...,...
2025-10-21,107.769997,108.480003,106.669998,107.900002,39905700
2025-10-22,104.599998,107.739998,101.580002,107.449997,67190400


In [32]:
# PREPARE PRICE SERIES

def select_close_series(df, ticker):
    if isinstance(df.columns, pd.MultiIndex):
        if ('Close', ticker) in df.columns:
            s = df[('Close', ticker)]
        else:
            cols = [c for c in df.columns if 'Close' in str(c)]
            if not cols:
                raise KeyError("Close not found")
            s = df[cols[0]]
    else:
        s = df['Close']
    return s.astype(float).squeeze()

close = select_close_series(stock_data, TICKER)
close.name = 'price'

# Simple split
split_idx = int(len(close) * TRAIN_RATIO)
train_close = close.iloc[:split_idx].copy()
val_close = close.iloc[split_idx:].copy()

print(f"Data ready: train={train_close.index[0].date()} → {train_close.index[-1].date()} | val={val_close.index[0].date()} → {val_close.index[-1].date()}")


Data ready: train=2010-02-11 → 2019-07-15 | val=2019-07-16 → 2025-10-27


# Aroon Indicator Strategy

This notebook optimizes an **Aroon indicator crossover strategy** for TQQQ trend following.

**Strategy Logic**: 
- BUY when Aroon Up crosses above Aroon Down
- SELL when Aroon Up crosses below Aroon Down

**Aroon Indicator**:
- Aroon Up: Measures time since highest high (0-100)
- Aroon Down: Measures time since lowest low (0-100)
- Length parameter: Lookback period for calculation


In [33]:
# Define Parameter Range for Aroon Optimization

param_bounds = {
    'aroon_length': (5, 100),  # Aroon period (lookback window)
}

print("Parameter Bounds for Optimization:")
print(f"  Aroon Length: {param_bounds['aroon_length'][0]} - {param_bounds['aroon_length'][1]} periods")
print("\n✅ Ready for optimization!")


Parameter Bounds for Optimization:
  Aroon Length: 5 - 100 periods

✅ Ready for optimization!


In [34]:
# Core Optimization Function for Aroon Strategy

def evaluate_aroon_strategy(aroon_length, data=train_close, return_metrics=False):
    """
    Evaluate Aroon crossover strategy.
    
    Args:
        aroon_length: Aroon lookback period
        data: price data to test on
        return_metrics: if True, return full metrics dict; if False, return negative Sortino ratio
    
    Returns:
        Negative Sortino ratio (for minimization) or full metrics dict
    """
    
    try:
        # Calculate Aroon using TA-Lib
        close_array = data.values
        aroon_up, aroon_down = talib.AROON(close_array, close_array, timeperiod=aroon_length)
        
        # Convert to Series
        aroon_up_series = pd.Series(aroon_up, index=data.index)
        aroon_down_series = pd.Series(aroon_down, index=data.index)
        
        # Generate crossover signals
        # BUY: Aroon Up crosses above Aroon Down
        entries = aroon_up_series.vbt.crossed_above(aroon_down_series)
        
        # SELL: Aroon Up crosses below Aroon Down  
        exits = aroon_up_series.vbt.crossed_below(aroon_down_series)
        
        # Convert to arrays
        entries_array = pd.Series(np.asarray(entries).ravel(), index=data.index, dtype=bool)
        exits_array = pd.Series(np.asarray(exits).ravel(), index=data.index, dtype=bool)
        
        # Backtest
        portfolio = vbt.Portfolio.from_signals(
            close=data,
            entries=entries_array,
            exits=exits_array,
            init_cash=100_000,
            fees=0.0005,
            slippage=0.0005,
            freq='D'
        )
        
        # Calculate key metrics
        sharpe_ratio = float(portfolio.sharpe_ratio(freq='D'))
        sortino_ratio = float(portfolio.sortino_ratio(freq='D'))
        total_return = float(portfolio.total_return())
        max_drawdown = float(portfolio.max_drawdown())
        volatility = float(portfolio.annualized_volatility(freq='D'))
        
        # Trade metrics
        trades = portfolio.trades
        total_trades = len(trades)
        
        # Check for sufficient trading activity
        years = max((data.index[-1] - data.index[0]).days / 365.25, 1e-9)
        trades_per_year = total_trades / years
        
        if trades_per_year < 1:  # Too few trades
            if return_metrics:
                return {
                    'aroon_length': aroon_length,
                    'sharpe_ratio': np.nan,
                    'sortino_ratio': np.nan,
                    'total_return': np.nan,
                    'annualized_return': np.nan,
                    'max_drawdown': np.nan,
                    'volatility': np.nan,
                    'total_trades': total_trades,
                    'win_rate': np.nan,
                    'profit_factor': np.nan,
                    'expectancy': np.nan,
                    'trades_per_year': trades_per_year
                }
            return 999.0
        
        if return_metrics:
            # Calculate additional metrics
            win_rate_pct = np.nan
            profit_factor = np.nan
            expectancy = 0.0
            annualized_return = (1.0 + total_return) ** (1.0 / years) - 1.0 if years > 0 else np.nan
            
            if total_trades > 0:
                tr = trades.returns.values if hasattr(trades.returns, 'values') else np.array(trades.returns)
                if tr.size > 0:
                    pos = tr[tr > 0]
                    neg = tr[tr < 0]
                    win_rate_pct = (len(pos) / len(tr)) * 100.0
                    gains = pos.sum() if len(pos) else 0.0
                    losses = abs(neg.sum()) if len(neg) else 0.0
                    profit_factor = (gains / losses) if losses > 0 else np.inf
                    expectancy = float(tr.mean())
            
            return {
                'aroon_length': aroon_length,
                'sharpe_ratio': sharpe_ratio,
                'sortino_ratio': sortino_ratio,
                'total_return': total_return,
                'annualized_return': annualized_return,
                'max_drawdown': max_drawdown,
                'volatility': volatility,
                'total_trades': total_trades,
                'win_rate': win_rate_pct,
                'profit_factor': profit_factor,
                'expectancy': expectancy,
                'trades_per_year': trades_per_year
            }
        else:
            # Return negative Sortino for minimization
            return -sortino_ratio
            
    except Exception as e:
        return 999.0  # Large penalty for errors

print("✅ Core optimization function created for Aroon strategy!")
print("Function evaluates Aroon Up/Down crossover signals.")


✅ Core optimization function created for Aroon strategy!
Function evaluates Aroon Up/Down crossover signals.


In [35]:
# Import optimization libraries
try:
    from skopt import gp_minimize
    from skopt.space import Real
    from skopt.utils import use_named_args
    SKOPT_AVAILABLE = True
except ImportError:
    SKOPT_AVAILABLE = False

try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False

print(f"scikit-optimize: {'✅' if SKOPT_AVAILABLE else '❌'}")
print(f"Optuna: {'✅' if OPTUNA_AVAILABLE else '❌'}")


scikit-optimize: ✅
Optuna: ✅


In [36]:
# OPTUNA OPTIMIZATION - Aroon Strategy

print("=" * 70)
print("OPTUNA OPTIMIZATION: Aroon Indicator Strategy")
print(f"Training Period: {train_close.index[0].date()} → {train_close.index[-1].date()}")
print("=" * 70)

optimization_results = []
evaluation_count = 0

def objective(trial):
    """Optuna objective function."""
    global evaluation_count
    evaluation_count += 1
    
    # Suggest parameter
    aroon_length = trial.suggest_int('aroon_length', param_bounds['aroon_length'][0], param_bounds['aroon_length'][1])
    
    # Evaluate strategy
    result = evaluate_aroon_strategy(aroon_length)
    
    # Debug output
    if result == 999.0 and evaluation_count <= 5:
        print(f"Trial {evaluation_count}: Aroon({aroon_length}) → No valid trades")
    elif evaluation_count % 10 == 0 or result != 999.0:
        print(f"Trial {evaluation_count}: Aroon({aroon_length}) → Sortino: {-result:.3f}")
    
    return result

if OPTUNA_AVAILABLE:
    # Create study
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=5)
    )
    
    print("\nStarting Optuna optimization...\n")
    
    study.optimize(objective, n_trials=1000, show_progress_bar=True)
    
    # Get best parameters
    best_params = study.best_params
    best_aroon_length = best_params['aroon_length']
    best_sortino = -study.best_value
    
    # Evaluate best parameters to get full metrics
    best_result = evaluate_aroon_strategy(
        best_aroon_length, 
        return_metrics=True
    )
    
    # Create results dataframe
    results_df = pd.DataFrame([best_result])
    
    print("\n✅ Optimization complete!")
    print(f"Best Parameters: Aroon({best_aroon_length})")
    print(f"Sortino Ratio: {best_sortino:.3f}")
    
else:
    print("❌ Optuna not available")


[I 2025-10-27 23:09:23,298] A new study created in memory with name: no-name-1db6be91-01de-4fb9-84a1-d074a7c0a1d4


OPTUNA OPTIMIZATION: Aroon Indicator Strategy
Training Period: 2010-02-11 → 2019-07-15

Starting Optuna optimization...



  0%|          | 0/1000 [00:00<?, ?it/s]

Trial 1: Aroon(40) → Sortino: 1.372
[I 2025-10-27 23:09:23,360] Trial 0 finished with value: -1.371693709167557 and parameters: {'aroon_length': 40}. Best is trial 0 with value: -1.371693709167557.
Trial 2: Aroon(96) → Sortino: 1.369
[I 2025-10-27 23:09:23,393] Trial 1 finished with value: -1.3685178025889926 and parameters: {'aroon_length': 96}. Best is trial 0 with value: -1.371693709167557.
Trial 3: Aroon(75) → Sortino: 2.016
[I 2025-10-27 23:09:23,426] Trial 2 finished with value: -2.0159801719317407 and parameters: {'aroon_length': 75}. Best is trial 2 with value: -2.0159801719317407.
Trial 4: Aroon(62) → Sortino: 2.107
[I 2025-10-27 23:09:23,460] Trial 3 finished with value: -2.1074612030454927 and parameters: {'aroon_length': 62}. Best is trial 3 with value: -2.1074612030454927.
Trial 5: Aroon(19) → Sortino: 1.008
[I 2025-10-27 23:09:23,489] Trial 4 finished with value: -1.0084250861370634 and parameters: {'aroon_length': 19}. Best is trial 3 with value: -2.1074612030454927.
Tri

In [37]:
# Display Results

if 'results_df' in globals() and not results_df.empty:
    best = results_df.iloc[0]
    
    print("OPTIMIZATION RESULTS:")
    print("=" * 50)
    print(f"Best Parameters:")
    print(f"  Aroon Length: {int(best['aroon_length'])}")
    
    print("\nPerformance Metrics:")
    print(f"  Total Return: {best['total_return']:.2%}")
    print(f"  Sharpe Ratio: {best['sharpe_ratio']:.3f}")
    print(f"  Sortino Ratio: {best['sortino_ratio']:.3f}")
    print(f"  Max Drawdown: {best['max_drawdown']:.2%}")
    print(f"  Total Trades: {int(best['total_trades'])}")
    print(f"  Win Rate: {best['win_rate']:.1f}%")
    print(f"  Profit Factor: {best['profit_factor']:.2f}")
else:
    print("⚠️ No results found. Run optimization first.")


OPTIMIZATION RESULTS:
Best Parameters:
  Aroon Length: 66

Performance Metrics:
  Total Return: 2905.28%
  Sharpe Ratio: 1.435
  Sortino Ratio: 2.193
  Max Drawdown: -50.06%
  Total Trades: 19
  Win Rate: 94.7%
  Profit Factor: 135.59


In [38]:
# BENCHMARK ANALYSIS: Best Strategy vs Market (Buy & Hold)

if 'results_df' in globals() and not results_df.empty:
    best = results_df.iloc[0]
    aroon_length = int(best['aroon_length'])

    print(f"BENCHMARK ANALYSIS: Best Aroon({aroon_length}) vs Buy & Hold")
    print("=" * 70)

    # Recreate the BEST strategy portfolio
    close_array = train_close.values
    aroon_up, aroon_down = talib.AROON(close_array, close_array, timeperiod=aroon_length)
    aroon_up_series = pd.Series(aroon_up, index=train_close.index)
    aroon_down_series = pd.Series(aroon_down, index=train_close.index)

    entries = aroon_up_series.vbt.crossed_above(aroon_down_series)
    exits = aroon_up_series.vbt.crossed_below(aroon_down_series)

    best_pf = vbt.Portfolio.from_signals(
        close=train_close,
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.0005,
        slippage=0.0005,
        freq='D'
    )

    # Create buy & hold benchmark
    benchmark_entries = np.zeros(len(train_close), dtype=bool)
    benchmark_entries[0] = True
    benchmark_exits = np.zeros(len(train_close), dtype=bool)

    benchmark_pf = vbt.Portfolio.from_signals(
        close=train_close,
        entries=benchmark_entries,
        exits=benchmark_exits,
        init_cash=100_000,
        fees=0.0005,
        freq='D'
    )

    # Strategy annualized return
    strategy_annualized_return = best['annualized_return']

    # Benchmark metrics
    bench_total_return = benchmark_pf.total_return()
    bench_annualized_return = benchmark_pf.annualized_return(freq='D')
    bench_sharpe = benchmark_pf.sharpe_ratio(freq='D')
    bench_sortino = benchmark_pf.sortino_ratio(freq='D')
    bench_max_drawdown = benchmark_pf.max_drawdown()
    bench_volatility = benchmark_pf.annualized_volatility(freq='D')

    print("\nPERFORMANCE COMPARISON:")
    print(f"Strategy (Aroon {aroon_length}):")
    print(f"  Total Return:      {best['total_return']:.2%}")
    print(f"  Annualized Return: {strategy_annualized_return:.2%}")
    print(f"  Sharpe Ratio:      {best['sharpe_ratio']:.3f}")
    print(f"  Sortino Ratio:     {best['sortino_ratio']:.3f}")
    print(f"  Max Drawdown:      {best['max_drawdown']:.2%}")
    print(f"  Volatility:        {best['volatility']:.2%}")
    print(f"  Total Trades:      {best['total_trades']}")
    print(f"  Win Rate:          {best['win_rate']:.1f}%")
    print(f"  Profit Factor:     {best['profit_factor']:.2f}")

    print(f"\nBenchmark (Buy & Hold {TICKER}):")
    print(f"  Total Return:      {bench_total_return:.2%}")
    print(f"  Annualized Return: {bench_annualized_return:.2%}")
    print(f"  Sharpe Ratio:      {bench_sharpe:.3f}")
    print(f"  Sortino Ratio:     {bench_sortino:.3f}")
    print(f"  Max Drawdown:      {bench_max_drawdown:.2%}")
    print(f"  Volatility:        {bench_volatility:.2%}")

    # Outperformance metrics
    excess_return = best['total_return'] - bench_total_return
    excess_annualized_return = strategy_annualized_return - bench_annualized_return
    sharpe_diff = best['sharpe_ratio'] - bench_sharpe
    sortino_diff = best['sortino_ratio'] - bench_sortino
    excess_max_drawdown = best['max_drawdown'] - bench_max_drawdown

    print(f"\nOUTPERFORMANCE METRICS:")
    print(f"  Excess Return:           {excess_return:.2%}")
    print(f"  Excess Annualized Return: {excess_annualized_return:.2%}")
    print(f"  Sharpe Difference:       {sharpe_diff:.3f}")
    print(f"  Sortino Difference:      {sortino_diff:.3f}")
    print(f"  Excess Max Drawdown:      {excess_max_drawdown:.2%} ({'Better' if excess_max_drawdown > 0 else '❌ Worse'})")
    
    # Calculate beta
    market_beta = best['volatility'] / bench_volatility if bench_volatility != 0 else np.nan
    print(f"  Market Beta (approx):     {market_beta:.3f} ({'Lower risk' if market_beta < 1 else '⚠️ Higher risk'})")
else:
    print("⚠️ No results available for benchmark analysis.")


BENCHMARK ANALYSIS: Best Aroon(66) vs Buy & Hold

PERFORMANCE COMPARISON:
Strategy (Aroon 66):
  Total Return:      2905.28%
  Annualized Return: 43.51%
  Sharpe Ratio:      1.435
  Sortino Ratio:     2.193
  Max Drawdown:      -50.06%
  Volatility:        42.95%
  Total Trades:      19.0
  Win Rate:          94.7%
  Profit Factor:     135.59

Benchmark (Buy & Hold TQQQ):
  Total Return:      3898.56%
  Annualized Return: 76.44%
  Sharpe Ratio:      1.232
  Sortino Ratio:     1.750
  Max Drawdown:      -58.08%
  Volatility:        61.63%

OUTPERFORMANCE METRICS:
  Excess Return:           -993.28%
  Excess Annualized Return: -32.94%
  Sharpe Difference:       0.203
  Sortino Difference:      0.442
  Excess Max Drawdown:      8.03% (Better)
  Market Beta (approx):     0.697 (Lower risk)


In [39]:
# IN-SAMPLE vs OUT-OF-SAMPLE COMPARISON

FREQ = "1D"

if 'results_df' in globals() and not results_df.empty:
    best = results_df.iloc[0]
    aroon_length = int(best['aroon_length'])

    # Out-of-sample validation
    close_array_val = val_close.values
    aroon_up_val, aroon_down_val = talib.AROON(close_array_val, close_array_val, timeperiod=aroon_length)
    aroon_up_series_val = pd.Series(aroon_up_val, index=val_close.index)
    aroon_down_series_val = pd.Series(aroon_down_val, index=val_close.index)

    entries_val = aroon_up_series_val.vbt.crossed_above(aroon_down_series_val)
    exits_val = aroon_up_series_val.vbt.crossed_below(aroon_down_series_val)

    pf_val = vbt.Portfolio.from_signals(
        close=val_close,
        entries=entries_val,
        exits=exits_val,
        init_cash=100_000,
        fees=0.0005,
        slippage=0.0005,
        freq=FREQ
    )

    # Out-of-sample metrics
    val_total_return = pf_val.total_return()
    val_annualized_return = pf_val.annualized_return(freq=FREQ)
    val_sharpe = pf_val.sharpe_ratio(freq=FREQ)
    val_sortino = pf_val.sortino_ratio(freq=FREQ)
    val_max_drawdown = pf_val.max_drawdown()
    val_volatility = pf_val.annualized_volatility(freq=FREQ)

    trades_val = pf_val.trades
    val_total_trades = len(trades_val)
    years = max((val_close.index[-1] - val_close.index[0]).days / 365.25, 1e-9)
    val_trades_per_year = val_total_trades / years

    val_win_rate_pct = np.nan
    val_profit_factor = np.nan
    val_expectancy = 0.0
    if val_total_trades > 0:
        tr = trades_val.returns.values if hasattr(trades_val.returns, 'values') else np.array(trades_val.returns)
        if tr.size > 0:
            pos = tr[tr > 0]
            neg = tr[tr < 0]
            val_win_rate_pct = (len(pos) / len(tr)) * 100 if len(tr) > 0 else np.nan
            gains = pos.sum() if len(pos) else 0.0
            losses = abs(neg.sum()) if len(neg) else 0.0
            val_profit_factor = gains / losses if losses > 0 else np.inf
            val_expectancy = tr.mean()

    def chg(before, after):
        if pd.isna(before) or pd.isna(after) or before == 0:
            return "N/A"
        return f"{((after - before) / abs(before)) * 100:+.1f}%"

    print(f"IN-SAMPLE vs OUT-OF-SAMPLE COMPARISON: Best Aroon({aroon_length})")
    print("=" * 80)

    print(f"{'METRIC':<25} {'IN-SAMPLE':<15} {'OUT-OF-SAMPLE':<15} {'DEGRADATION':<15}")

    print("\nRETURN METRICS:")
    print(f"{'Total Return':<25} {best['total_return']:<15.2%} {val_total_return:<15.2%} {chg(best['total_return'], val_total_return)}")
    print(f"{'Annualized Return':<25} {best['annualized_return']:<15.2%} {val_annualized_return:<15.2%} {chg(best['annualized_return'], val_annualized_return)}")

    print("\nRISK-ADJUSTED METRICS:")
    print(f"{'Sharpe Ratio':<25} {best['sharpe_ratio']:<15.3f} {val_sharpe:<15.3f} {chg(best['sharpe_ratio'], val_sharpe)}")
    print(f"{'Sortino Ratio':<25} {best['sortino_ratio']:<15.3f} {val_sortino:<15.3f} {chg(best['sortino_ratio'], val_sortino)}")

    print("\nRISK METRICS:")
    print(f"{'Max Drawdown':<25} {best['max_drawdown']:<15.2%} {val_max_drawdown:<15.2%} {chg(best['max_drawdown'], val_max_drawdown)}")
    print(f"{'Volatility':<25} {best['volatility']:<15.2%} {val_volatility:<15.2%} {chg(best['volatility'], val_volatility)}")

    print("\nTRADE METRICS:")
    print(f"{'Total Trades':<25} {best['total_trades']:<15.0f} {val_total_trades:<15.0f} {chg(best['total_trades'], val_total_trades)}")
    print(f"{'Trades per Year':<25} {best['trades_per_year']:<15.1f} {val_trades_per_year:<15.1f} {chg(best['trades_per_year'], val_trades_per_year)}")
    print(f"{'Win Rate %':<25} {best['win_rate']:<15.1f} {val_win_rate_pct:<15.1f} {chg(best['win_rate'], val_win_rate_pct)}")
    print(f"{'Profit Factor':<25} {best['profit_factor']:<15.2f} {val_profit_factor:<15.2f} {chg(best['profit_factor'], val_profit_factor)}")
    print(f"{'Expectancy':<25} {best['expectancy']:<15.4f} {val_expectancy:<15.4f} {chg(best['expectancy'], val_expectancy)}")
else:
    print("⚠️ No results for comparison.")


IN-SAMPLE vs OUT-OF-SAMPLE COMPARISON: Best Aroon(66)
METRIC                    IN-SAMPLE       OUT-OF-SAMPLE   DEGRADATION    

RETURN METRICS:
Total Return              2905.28%        341.09%         -88.3%
Annualized Return         43.51%          40.86%          -6.1%

RISK-ADJUSTED METRICS:
Sharpe Ratio              1.435           0.846           -41.1%
Sortino Ratio             2.193           1.254           -42.8%

RISK METRICS:
Max Drawdown              -50.06%         -66.11%         -32.1%
Volatility                42.95%          68.41%          +59.3%

TRADE METRICS:
Total Trades              19              11              -42.1%
Trades per Year           2.0             1.8             -13.2%
Win Rate %                94.7            81.8            -13.6%
Profit Factor             135.59          4.22            -96.9%
Expectancy                0.2014          0.1890          -6.2%


In [40]:
# COMPREHENSIVE VISUALIZATION

if 'results_df' in globals() and not results_df.empty:
    best = results_df.iloc[0]
    aroon_length = int(best['aroon_length'])

    # Calculate Aroon for full sample
    close_array_full = close.values
    aroon_up_full, aroon_down_full = talib.AROON(close_array_full, close_array_full, timeperiod=aroon_length)
    aroon_up_series_full = pd.Series(aroon_up_full, index=close.index)
    aroon_down_series_full = pd.Series(aroon_down_full, index=close.index)

    # Generate signals
    entries_full = aroon_up_series_full.vbt.crossed_above(aroon_down_series_full)
    exits_full = aroon_up_series_full.vbt.crossed_below(aroon_down_series_full)

    # Create portfolio
    pf_full = vbt.Portfolio.from_signals(
        close=close,
        entries=entries_full,
        exits=exits_full,
        init_cash=100_000,
        fees=0.0005,
        slippage=0.0005,
        freq='D'
    )

    # Get returns
    ret = pf_full.returns()

    # Create comprehensive figure
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(4, 2, hspace=0.5, wspace=0.3, 
                          height_ratios=[2.5, 1.5, 1.5, 1],
                          left=0.05, right=0.95, top=0.95, bottom=0.05)

    fig.suptitle(f'Aroon({aroon_length}) Trading Strategy - Comprehensive Analysis', 
                 fontsize=16, fontweight='bold', y=0.98)

    # 1. Price + Aroon lines + Signals
    ax1 = fig.add_subplot(gs[0, :])
    ax1_twin = ax1.twinx()

    ax1.plot(close.index, close.values, label='Close Price', color='black', linewidth=1.5, alpha=0.7)
    ax1_twin.plot(close.index, aroon_up_series_full.values, label='Aroon Up', color='green', linewidth=1.2, alpha=0.8)
    ax1_twin.plot(close.index, aroon_down_series_full.values, label='Aroon Down', color='red', linewidth=1.2, alpha=0.8)
    
    # Mark buy/sell signals
    buy_indices = entries_full[entries_full].index
    sell_indices = exits_full[exits_full].index
    if len(buy_indices) > 0:
        ax1.scatter(buy_indices, close.reindex(buy_indices).values, marker='^', color='green', s=100, label='Buy', zorder=5)
    if len(sell_indices) > 0:
        ax1.scatter(sell_indices, close.reindex(sell_indices).values, marker='v', color='red', s=100, label='Sell', zorder=5)

    ax1.set_title('Price, Aroon Indicator, and Trading Signals', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Price ($)', fontsize=10)
    ax1_twin.set_ylabel('Aroon (0-100)', fontsize=10)
    ax1.legend(loc='upper left', fontsize=9)
    ax1_twin.legend(loc='upper right', fontsize=9)
    ax1.grid(True, alpha=0.3)

    # 2. Equity Curve
    ax2 = fig.add_subplot(gs[1, :])
    equity = (1 + ret).cumprod()
    ax2.plot(close.index, equity.values, color='black', linewidth=2, label='Equity Curve')
    ax2.axvline(x=train_close.index[-1], color='red', linestyle='--', linewidth=2, alpha=0.6, label='Train/Val Split')
    ax2.set_title('Equity Curve', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Cumulative Returns', fontsize=10)
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3)

    # 3. Drawdown
    ax3 = fig.add_subplot(gs[2, :])
    peak = equity.cummax()
    drawdown = (equity - peak) / peak
    ax3.fill_between(close.index, drawdown * 100, 0, color='red', alpha=0.5)
    ax3.plot(close.index, drawdown * 100, color='darkred', linewidth=1.5)
    ax3.set_title('Drawdown Chart', fontsize=13, fontweight='bold')
    ax3.set_ylabel('Drawdown (%)', fontsize=10)
    ax3.set_xlabel('Date', fontsize=10)
    ax3.grid(True, alpha=0.3)

    # 4. Rolling Sharpe
    ax4 = fig.add_subplot(gs[3, :])
    rolling_window = max(20, min(252, max(1, len(ret) // 4)))
    if len(ret) > rolling_window:
        rolling_sharpe = ret.rolling(window=rolling_window).apply(
            lambda x: (x.mean() * 252) / (x.std() * np.sqrt(252)) if x.std() and x.std() != 0 else np.nan,
            raw=False
        )
        ax4.plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=2, color='blue', alpha=0.85)
        ax4.axhline(y=1.0, color='green', linestyle='--', alpha=0.6, label='Sharpe = 1.0')
        ax4.axhline(y=0.5, color='orange', linestyle='--', alpha=0.6, label='Sharpe = 0.5')
        ax4.axvline(x=train_close.index[-1], color='purple', linestyle='--', linewidth=2, alpha=0.6)

    ax4.set_title(f'Rolling Sharpe Ratio (window={rolling_window})', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Sharpe Ratio', fontsize=10)
    ax4.set_xlabel('Date', fontsize=10)
    ax4.legend(loc='upper left', fontsize=9)
    ax4.grid(True, alpha=0.3)
    ax4.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m'))

    plt.savefig('Aroon_Comprehensive_Analysis.png', dpi=300, bbox_inches='tight')
    print("✅ Comprehensive visualization saved as 'Aroon_Comprehensive_Analysis.png'")
    plt.close(fig)

    print(f"\nTraining Period: {train_close.index[0].date()} to {train_close.index[-1].date()} ({len(train_close)} days)")
    print(f"Validation Period: {val_close.index[0].date()} to {val_close.index[-1].date()} ({len(val_close)} days)")
    print(f"\nBest Parameters: Aroon({aroon_length})")
else:
    print("⚠️ No results available for comprehensive visualization.")


✅ Comprehensive visualization saved as 'Aroon_Comprehensive_Analysis.png'

Training Period: 2010-02-11 to 2019-07-15 (2371 days)
Validation Period: 2019-07-16 to 2025-10-27 (1581 days)

Best Parameters: Aroon(66)


In [41]:
# METRICS TABLE - In-Sample vs Out-of-Sample

if 'results_df' in globals() and not results_df.empty:
    best = results_df.iloc[0]
    aroon_length = int(best['aroon_length'])

    # Get train and validation results
    train_result = evaluate_aroon_strategy(aroon_length, data=train_close, return_metrics=True)
    
    val_result = evaluate_aroon_strategy(aroon_length, data=val_close, return_metrics=True)
    
    # Calculate trades per year
    train_trades_per_year = train_result['total_trades'] / (len(train_close) / 252)
    val_trades_per_year = val_result['total_trades'] / (len(val_close) / 252)
    
    # Create separate figure for metrics table
    fig2 = plt.figure(figsize=(14, 10))
    ax = fig2.add_subplot(111)
    ax.axis('off')
    
    # Create metrics table
    table_data = [
        ['Metric', 'In-Sample (Training)', 'Out-of-Sample (Validation)'],
        ['Period', f'{train_close.index[0].date()} to {train_close.index[-1].date()}', 
         f'{val_close.index[0].date()} to {val_close.index[-1].date()}'],
        ['Annualized Return', f"{train_result['annualized_return']:.2%}", f"{val_result['annualized_return']:.2%}"],
        ['Sharpe Ratio', f"{train_result['sharpe_ratio']:.3f}", f"{val_result['sharpe_ratio']:.3f}"],
        ['Sortino Ratio', f"{train_result['sortino_ratio']:.3f}", f"{val_result['sortino_ratio']:.3f}"],
        ['Max Drawdown', f"{train_result['max_drawdown']:.2%}", f"{val_result['max_drawdown']:.2%}"],
        ['Trades per Year', f"{train_trades_per_year:.2f}", f"{val_trades_per_year:.2f}"],
        ['Win Rate', f"{train_result['win_rate']:.1f}%", f"{val_result['win_rate']:.1f}%"],
        ['Profit Factor', f"{train_result['profit_factor']:.2f}", f"{val_result['profit_factor']:.2f}"]
    ]
    
    table = ax.table(cellText=table_data, cellLoc='left', loc='center',
                     colWidths=[0.4, 0.3, 0.3], bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2.5)
    
    # Style the table
    for i in range(len(table_data)):
        for j in range(len(table_data[0])):
            cell = table[(i, j)]
            if i == 0:
                cell.set_facecolor('#4CAF50')
                cell.set_text_props(weight='bold', color='white')
            else:
                if j == 0:
                    cell.set_facecolor('#F0F0F0')
                else:
                    cell.set_facecolor('#FFFFFF')
    
    fig2.suptitle(f'Performance Metrics Summary - Aroon({aroon_length})', fontsize=16, fontweight='bold', y=0.95)
    plt.savefig('Aroon_Metrics_Table.png', dpi=300, bbox_inches='tight')
    print("✅ Metrics table saved as 'Aroon_Metrics_Table.png'")
    plt.close(fig2)
    
    print(f"\n📊 Metrics Summary:")
    print(f"Training: {train_result['total_return']:.2%} return, {train_result['sharpe_ratio']:.3f} Sharpe")
    print(f"Validation: {val_result['total_return']:.2%} return, {val_result['sharpe_ratio']:.3f} Sharpe")
else:
    print("⚠️ No results available for metrics table.")


✅ Metrics table saved as 'Aroon_Metrics_Table.png'

📊 Metrics Summary:
Training: 2905.28% return, 1.435 Sharpe
Validation: 341.09% return, 0.846 Sharpe


In [42]:
# PARAMETER SENSITIVITY ANALYSIS (±15 around best aroon_length)

if 'results_df' in globals() and not results_df.empty:
    best = results_df.iloc[0]
    aroon_length = int(best['aroon_length'])

    print(f"🔬 Parameter Sensitivity Analysis for Best Aroon({aroon_length})")
    print("=" * 80)

    # Create sensitivity ranges (±15 around best length)
    candidates = list(range(max(5, aroon_length - 15), min(101, aroon_length + 16)))

    price_np = train_close.to_numpy(dtype=float)
    idx = train_close.index

    def eval_combo(length: int) -> dict:
        aroon_up, aroon_down = talib.AROON(price_np, price_np, timeperiod=length)
        aroon_up_series = pd.Series(aroon_up, index=idx)
        aroon_down_series = pd.Series(aroon_down, index=idx)

        entries = aroon_up_series.vbt.crossed_above(aroon_down_series)
        exits = aroon_up_series.vbt.crossed_below(aroon_down_series)

        pf = vbt.Portfolio.from_signals(
            close=price_np, entries=entries, exits=exits,
            init_cash=100_000, fees=0.0005, slippage=0.0005, freq='1D'
        )

        total_return = float(pf.total_return())
        sharpe = float(pf.sharpe_ratio(freq='D'))
        sortino = float(pf.sortino_ratio(freq='D'))
        mdd = float(pf.max_drawdown())
        vol = float(pf.annualized_volatility(freq='D'))

        trades = pf.trades
        ntr = len(trades)
        win_rate_pct = np.nan
        profit_factor = np.nan
        expectancy = 0.0
        if ntr > 0:
            tr = trades.returns.values if hasattr(trades.returns, 'values') else np.array(trades.returns)
            if tr.size > 0:
                pos = tr[tr > 0]
                neg = tr[tr < 0]
                win_rate_pct = (len(pos) / len(tr)) * 100 if len(tr) else np.nan
                gains = pos.sum() if len(pos) else 0.0
                losses = abs(neg.sum()) if len(neg) else 0.0
                profit_factor = gains / losses if losses > 0 else np.inf
                expectancy = float(tr.mean())

        return {
            'aroon_length': length,
            'sharpe': sharpe, 'sortino': sortino,
            'total_return': total_return, 'max_drawdown': mdd, 'volatility': vol,
            'total_trades': ntr, 'win_rate_pct': win_rate_pct,
            'profit_factor': profit_factor, 'expectancy': expectancy
        }

    rows = []
    for length in candidates:
        try:
            rows.append(eval_combo(length))
        except Exception:
            pass

    if not rows:
        print("⚠️ No sensitivity results computed.")
    else:
        sens = pd.DataFrame(rows)

        # Show as a table
        cols = ['aroon_length','sharpe','sortino','total_return','max_drawdown','volatility',
                'total_trades','win_rate_pct','profit_factor','expectancy']
        sens_table = sens[cols].sort_values('aroon_length')

        print(f"\n📊 Sensitivity Results ({len(sens_table)} variations tested):\n")
        print(sens_table.to_string())

        # Compact variation summary
        metric_cols = ['sharpe','sortino','total_return','max_drawdown','volatility',
                       'win_rate_pct','profit_factor','expectancy']
        summary = sens_table[metric_cols].agg(['mean','std','min','max']).T

        print("\n📈 Sensitivity Summary (mean / std / min / max):")
        print(summary.round(4).to_string())

        # Highlight best performer in sensitivity test
        best_sens_idx = sens_table['sharpe'].idxmax()
        best_sens = sens_table.loc[best_sens_idx]
        print(f"\n🏆 Best in Sensitivity Test: Aroon({int(best_sens['aroon_length'])}) → Sharpe: {best_sens['sharpe']:.3f}")
else:
    print("⚠️ No results available for sensitivity analysis.")


🔬 Parameter Sensitivity Analysis for Best Aroon(66)

📊 Sensitivity Results (31 variations tested):

    aroon_length    sharpe   sortino  total_return  max_drawdown  volatility  total_trades  win_rate_pct  profit_factor  expectancy
0             51  1.067436  1.585114     10.038776     -0.500575    0.435493            21     85.714286       7.994250    0.132313
1             52  1.112617  1.660206     11.449403     -0.500575    0.433742            21     85.714286       9.720421    0.137610
2             53  1.120280  1.669128     11.652048     -0.500575    0.432561            20     85.000000      13.097240    0.144013
3             54  1.137128  1.696652     12.244641     -0.500575    0.432221            20     85.000000      11.446414    0.147774
4             55  1.200985  1.797987     14.829122     -0.500575    0.432041            20     85.000000      18.626312    0.156333
5             56  1.248461  1.876819     17.193396     -0.500575    0.433165            19     89.473684    

c:\Users\harka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

